In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

import warnings
warnings.filterwarnings('ignore')

# Loading of dataset

In [2]:
from cesium import datasets

def loadDataset():
    eeg = datasets.fetch_andrzejak()
    
    # Group together classes (Z, O), (N, F), (S) as normal, interictal, ictal
    eeg["classes"] = eeg["classes"].astype("U16") #  allocate memory for longer class names
    eeg["classes"][np.logical_or(eeg["classes"]=="Z", eeg["classes"]=="O")] = "Normal"
    eeg["classes"][np.logical_or(eeg["classes"]=="N", eeg["classes"]=="F")] = "Interictal"
    eeg["classes"][eeg["classes"]=="S"] = "Ictal"
    
    return eeg

In [3]:
def describeEEGDataset(eeg):
    fig, ax = plt.subplots(len(np.unique(eeg["classes"])), 1, sharey=True, figsize=(16,10))
    for label, subplot in zip(np.unique(eeg["classes"]), ax):
        i = np.where(eeg["classes"] == label)[0][0]
        subplot.plot(eeg["times"][i], eeg["measurements"][i])
        subplot.set(xlabel="time (s)", ylabel="signal", title=label)

# Adding of noise

In [4]:
from scipy.signal import chirp,sawtooth

In [5]:
def addNoiseToEEG(eeg, freq, amp):
    # Add noise
    mu, sigma = 0, 3.3 # mean and standard deviation
    x = len(eeg["times"][0])
    
    noised = eeg["measurements"].copy()

    for i in range(0, 500):
        noised[i] = eeg["measurements"][i] + \
        amp * np.random.normal(mu, sigma, x) + \
        amp * chirp(eeg["times"][i], f0=freq, f1=1, t1=10, method='linear') + \
        amp * sawtooth(2 * np.pi * freq * eeg["times"][i])
        
    return noised

# Classifiers

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Featurization functions

In [7]:
import numpy as np
import scipy.stats
from cesium import featurize
import pywt
from cesium import featurize

features_to_use = ["amplitude",
                   "percent_beyond_1_std",
                   "maximum",
                   "max_slope",
                   "median",
                   "median_absolute_deviation",
                   "percent_close_to_median",
                   "minimum",
                   "skew",
                   "std",
                   "weighted_average"]

# guo
def mean_signal(t, m, e):
    return np.mean(m)

def std_signal(t, m, e):
    return np.std(m)

def mean_square_signal(t, m, e):
    return np.mean(m ** 2)

def abs_diffs_signal(t, m, e):
    return np.sum(np.abs(np.diff(m)))

def skew_signal(t, m, e):
    return scipy.stats.skew(m)

%run custom_methods.ipynb

def hurst_signal(t, m, e):
    return hurst(m)

guo_features = {
    "mean": mean_signal,
    "std": std_signal,
    "mean2": mean_square_signal,
    "abs_diffs": abs_diffs_signal,
    "skew": skew_signal,
    "hurst": hurst_signal
}

In [8]:
def runSimpleClassifier(eeg):
    fset = featurize.featurize_time_series(times=eeg["times"],
                                              values=eeg["noisy_measurements"],
                                              errors=None,
                                              features_to_use=features_to_use)
    
    # Model building
    train, test = train_test_split(np.arange(len(eeg["classes"])), random_state=17)
    
    model = RandomForestClassifier(n_estimators=128, max_features="auto",
                                      random_state=0)
    model.fit(fset.iloc[train], eeg["classes"][train])
    
    # prediction
    preds = model.predict(fset)
    
    train_r = accuracy_score(preds[train], eeg["classes"][train])
    test_r = accuracy_score(preds[test], eeg["classes"][test])
    
    return train_r, test_r

In [9]:
def runGuoClassifier(eeg):
    # Featurize
    # guo
    n_channels = 5
    eeg["dwts"] = [pywt.wavedec(m, pywt.Wavelet("db1"), level=n_channels-1)
                   for m in eeg["noisy_measurements"]]
    fset = featurize.featurize_time_series(times=None, values=eeg["dwts"], errors=None,
                                           features_to_use=list(guo_features.keys()),
                                           custom_functions=guo_features)
    
    # Model building
    train, test = train_test_split(np.arange(len(eeg["classes"])), random_state=17)

    model = KNeighborsClassifier(3)
    model.fit(fset.iloc[train], eeg["classes"][train])

    # Prediction
    preds = model.predict(fset)
    
    train_r = accuracy_score(preds[train], eeg["classes"][train])
    test_r = accuracy_score(preds[test], eeg["classes"][test])
    
    return train_r, test_r